In [1]:
import numpy as np
import pandas as pd

In [19]:
heart = pd.read_csv("C:/Users/Kavita manoj/Downloads/Heart.csv", index_col='Unnamed: 0', na_values='NA')

array([[63, 1, 'typical', ..., 0.0, 'fixed', 'No'],
       [67, 1, 'asymptomatic', ..., 3.0, 'normal', 'Yes'],
       [67, 1, 'asymptomatic', ..., 2.0, 'reversable', 'Yes'],
       ...,
       [57, 1, 'asymptomatic', ..., 1.0, 'reversable', 'Yes'],
       [57, 0, 'nontypical', ..., 1.0, 'normal', 'Yes'],
       [38, 1, 'nonanginal', ..., nan, 'normal', 'No']], dtype=object)

In [3]:
heart.Ca = heart.Ca.fillna(heart.Ca.mode(dropna=True)[0])
heart.Thal = heart.Ca.fillna(heart.Thal.mode(dropna=True)[0])

In [4]:
heart.Sex = heart.Sex.astype(str)
heart.Fbs = heart.Fbs.astype(str)
heart.RestECG = heart.RestECG.astype(str)
heart.ExAng = heart.ExAng.astype(str)
heart.Ca = heart.Ca.astype(str)

In [5]:
from sklearn.preprocessing import scale

In [6]:
heart.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 303 entries, 1 to 303
Data columns (total 14 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   Age        303 non-null    int64  
 1   Sex        303 non-null    object 
 2   ChestPain  303 non-null    object 
 3   RestBP     303 non-null    int64  
 4   Chol       303 non-null    int64  
 5   Fbs        303 non-null    object 
 6   RestECG    303 non-null    object 
 7   MaxHR      303 non-null    int64  
 8   ExAng      303 non-null    object 
 9   Oldpeak    303 non-null    float64
 10  Slope      303 non-null    int64  
 11  Ca         303 non-null    object 
 12  Thal       303 non-null    float64
 13  AHD        303 non-null    object 
dtypes: float64(2), int64(5), object(7)
memory usage: 35.5+ KB


In [7]:
heart['Age_s'] = scale(heart.Age, with_mean=True, with_std=True)

In [8]:
heart['RestBP_s'] = scale(heart.RestBP, with_mean=True, with_std=True)

In [9]:
heart['Chol_s'] = scale(heart.Chol, with_mean=True, with_std=True)

In [10]:
heart['MaxHR_s'] = scale(heart.MaxHR, with_mean=True, with_std=True)

In [11]:
heart['OldPeak_s'] = scale(heart.Oldpeak, with_mean=True, with_std=True)
heart.head()

,Age,Sex,ChestPain,RestBP,Chol,Fbs,RestECG,MaxHR,ExAng,Oldpeak,Slope,Ca,Thal,AHD,Age_s,RestBP_s,Chol_s,MaxHR_s,OldPeak_s
1,63,1,typical,145,233,1,2,150,0,2.3,3,0.0,0.0,No,0.948726,0.757525,-0.264900,0.017197,1.087338
2,67,1,asymptomatic,160,286,0,2,108,1,1.5,2,3.0,3.0,Yes,1.392002,1.611220,0.760415,-1.821905,0.397182
3,67,1,asymptomatic,120,229,0,2,129,1,2.6,2,2.0,2.0,Yes,1.392002,-0.665300,-0.342283,-0.902354,1.346147
4,37,1,nonanginal,130,250,0,0,187,0,3.5,3,0.0,0.0,No,-1.932564,-0.096170,0.063974,1.637359,2.122573
5,41,0,nontypical,130,204,0,2,172,0,1.4,1,0.0,0.0,No,-1.489288,-0.096170,-0.825922,0.980537,0.310912


In [12]:
from statsmodels.tools.tools import add_constant
from statsmodels.stats.outliers_influence import variance_inflation_factor

In [13]:
heartQuant = add_constant(pd.DataFrame({'Age_s': heart.Age_s, 'Chol_s': heart.Chol_s, 'MaxHR_s': heart.MaxHR_s, 'OldPeak_s': heart.OldPeak_s, 'RestBP_s': heart.RestBP_s}))

In [14]:
vif = pd.DataFrame()
vif["Feature"] = heartQuant.columns
vif["VIF"] = [variance_inflation_factor(heartQuant.values, i) for i in range(len(heartQuant.columns))]

In [15]:
from statsmodels.formula.api import logit
heart.AHD = heart.AHD.map({'No': 0, 'Yes': 1})
heart.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 303 entries, 1 to 303
Data columns (total 19 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   Age        303 non-null    int64  
 1   Sex        303 non-null    object 
 2   ChestPain  303 non-null    object 
 3   RestBP     303 non-null    int64  
 4   Chol       303 non-null    int64  
 5   Fbs        303 non-null    object 
 6   RestECG    303 non-null    object 
 7   MaxHR      303 non-null    int64  
 8   ExAng      303 non-null    object 
 9   Oldpeak    303 non-null    float64
 10  Slope      303 non-null    int64  
 11  Ca         303 non-null    object 
 12  Thal       303 non-null    float64
 13  AHD        303 non-null    int64  
 14  Age_s      303 non-null    float64
 15  RestBP_s   303 non-null    float64
 16  Chol_s     303 non-null    float64
 17  MaxHR_s    303 non-null    float64
 18  OldPeak_s  303 non-null    float64
dtypes: float64(7), int64(6), object(6)
memory usage: 4

In [16]:
fit = logit('AHD~MaxHR_s+OldPeak_s+RestBP_s+Ca+Sex+ChestPain+Fbs+RestECG', data=heart).fit()

Optimization terminated successfully.
         Current function value: 0.356052
         Iterations 7


In [17]:
fit.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:                    AHD   No. Observations:                  303
Model:                          Logit   Df Residuals:                      289
Method:                           MLE   Df Model:                           13
Date:                Mon, 10 Apr 2023   Pseudo R-squ.:                  0.4838
Time:                        20:26:34   Log-Likelihood:                -107.88
converged:                       True   LL-Null:                       -208.99
Covariance Type:            nonrobust   LLR p-value:                 4.795e-36
===========================================================================================
                              coef    std err          z      P>|z|      [0.025      0.975]
-------------------------------------------------------------------------------------------
Intercept                  -1.2435      0.434     -2.868      0.004      -2.093      -0.394
Ca[T.1.0]                   1.9000      0.444      4.280      0.000       1.030       2.770
Ca[T.2.0]                   2.3963      0.626      3.825      0.000       1.168       3.624
Ca[T.3.0]                   1.8002      0.790      2.280      0.023       0.252       3.348
Sex[T.1]                    1.7424      0.419      4.157      0.000       0.921       2.564
ChestPain[T.nonanginal]    -2.2687      0.451     -5.028      0.000      -3.153      -1.384
ChestPain[T.nontypical]    -1.6123      0.525     -3.073      0.002      -2.641      -0.584
ChestPain[T.typical]       -2.6452      0.660     -4.008      0.000      -3.939      -1.352
Fbs[T.1]                   -0.2117      0.482     -0.439      0.661      -1.157       0.734
RestECG[T.1]                0.5536      2.039      0.271      0.786      -3.444       4.551
RestECG[T.2]                0.4877      0.353      1.382      0.167      -0.204       1.179
MaxHR_s                    -0.6111      0.218     -2.807      0.005      -1.038      -0.184
OldPeak_s                   0.7821      0.231      3.389      0.001       0.330       1.234
RestBP_s                    0.4833      0.184      2.632      0.008       0.123       0.843
===========================================================================================
"""

In [18]:
fit.predict()

array([0.33977533, 0.99323693, 0.99067445, 0.23925256, 0.05893743,
       0.08656516, 0.9650589 , 0.0979699 , 0.95902973, 0.90459325,
       0.58393148, 0.11359581, 0.56627338, 0.05939605, 0.17242955,
       0.1764215 , 0.09719183, 0.63589635, 0.02101493, 0.11620245,
       0.16909996, 0.03025416, 0.32911236, 0.87592745, 0.99121123,
       0.02459878, 0.00585644, 0.20111199, 0.6771505 , 0.87983685,
       0.31259109, 0.90416936, 0.07818737, 0.48026228, 0.04589825,
       0.32437447, 0.92002879, 0.98369714, 0.95197487, 0.23735004,
       0.92230964, 0.08059384, 0.39079277, 0.21456579, 0.11684936,
       0.65697156, 0.1244484 , 0.95776201, 0.18738171, 0.18300957,
       0.05297746, 0.50072828, 0.82532076, 0.08310259, 0.93966634,
       0.98948816, 0.42648451, 0.36912417, 0.50145505, 0.72293847,
       0.27325774, 0.05864655, 0.98140649, 0.00753975, 0.96443331,
       0.99417827, 0.53119175, 0.30741591, 0.97996506, 0.62975538,
       0.04789496, 0.82842817, 0.98835214, 0.90594597, 0.70173